<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Objective" data-toc-modified-id="Objective-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Objective</a></span></li><li><span><a href="#Used-Python-Libraries" data-toc-modified-id="Used-Python-Libraries-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Used Python Libraries</a></span></li><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Load Data</a></span><ul class="toc-item"><li><span><a href="#Chats-Data" data-toc-modified-id="Chats-Data-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Chats Data</a></span></li><li><span><a href="#Talking-Points-Data" data-toc-modified-id="Talking-Points-Data-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Talking Points Data</a></span></li><li><span><a href="#Merging-Data----Do-not-do-this-for-now-since-we-end-up-with-few-exemplars" data-toc-modified-id="Merging-Data----Do-not-do-this-for-now-since-we-end-up-with-few-exemplars-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Merging Data -- Do not do this for now since we end up with few exemplars</a></span></li></ul></li><li><span><a href="#A-First-Glance-at-Transcripts" data-toc-modified-id="A-First-Glance-at-Transcripts-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>A First Glance at Transcripts</a></span></li><li><span><a href="#A-First-Glance-at-Chats" data-toc-modified-id="A-First-Glance-at-Chats-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>A First Glance at Chats</a></span></li><li><span><a href="#Pre-Trained-Sentence-Encoder" data-toc-modified-id="Pre-Trained-Sentence-Encoder-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Pre-Trained Sentence Encoder</a></span><ul class="toc-item"><li><span><a href="#Example-of-Using-Encoder" data-toc-modified-id="Example-of-Using-Encoder-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Example of Using Encoder</a></span></li></ul></li><li><span><a href="#Data-Processing" data-toc-modified-id="Data-Processing-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Data Processing</a></span><ul class="toc-item"><li><span><a href="#Processing-Call-Transcripts" data-toc-modified-id="Processing-Call-Transcripts-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Processing Call Transcripts</a></span><ul class="toc-item"><li><span><a href="#Example-of-Processed-Transcript-Data" data-toc-modified-id="Example-of-Processed-Transcript-Data-7.1.1"><span class="toc-item-num">7.1.1&nbsp;&nbsp;</span>Example of Processed Transcript Data</a></span></li></ul></li><li><span><a href="#Processing-Chats" data-toc-modified-id="Processing-Chats-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>Processing Chats</a></span><ul class="toc-item"><li><span><a href="#Example-of-Processed-Chat-Data" data-toc-modified-id="Example-of-Processed-Chat-Data-7.2.1"><span class="toc-item-num">7.2.1&nbsp;&nbsp;</span>Example of Processed Chat Data</a></span></li></ul></li></ul></li><li><span><a href="#Analyzing-Processed-Data" data-toc-modified-id="Analyzing-Processed-Data-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Analyzing Processed Data</a></span><ul class="toc-item"><li><span><a href="#Analyzing-Transcript" data-toc-modified-id="Analyzing-Transcript-8.1"><span class="toc-item-num">8.1&nbsp;&nbsp;</span>Analyzing Transcript</a></span></li><li><span><a href="#Analyzing-Chats" data-toc-modified-id="Analyzing-Chats-8.2"><span class="toc-item-num">8.2&nbsp;&nbsp;</span>Analyzing Chats</a></span></li></ul></li><li><span><a href="#Interactive-Visualization-of-Results" data-toc-modified-id="Interactive-Visualization-of-Results-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Interactive Visualization of Results</a></span></li></ul></div>

## Objective

This project will prototype a screening tool to:

    * identify calls where customer agents speak about offers/recommendations they are able to see as talking points in the customer panel during a call;
    * determine whether the "discussed" botton (supposedly pressed by agents) describes accurately if a recommendation was pitched or not.
    
Two data tables will be used:

    * a table containing information about the call and the call transcripts 
    * a table containing information about discussion of talking points
    
<img src="xxx.png" width=800 /> ![](one_pager_img)

[Back to Contents](#Table-of-Contents)

## Used Python Libraries

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text #needed to avoid crashes
import os

In [2]:
import os
import glob

In [3]:
import pandas as pd
import numpy as np

In [4]:
from datetime import timedelta

In [5]:
import re

In [6]:
from sentence_transformers import SentenceTransformer, util

In [7]:
import psutil
import ray
import sys

In [8]:
from random import sample 

In [9]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

[Back to Contents](#Table-of-Contents)

## Load Data

We use data downloaded from Kaggle ([Ubuntu Dialogue Corpus](https://www.kaggle.com/rtatman/ubuntu-dialogue-corpus?ref=hackernoon.com) project).

In [ ]:
#https://registry.opendata.aws/topical-chat-enriched/

import boto3
import os
from botocore import UNSIGNED
from botocore.client import Config

def download_all_files():
    #initiate s3 resource
    s3 = boto3.resource('s3', config=Config(signature_version=UNSIGNED))
    # select bucket
    my_bucket = s3.Bucket('enriched-topical-chat')
    # download file into current directory
    for s3_object in my_bucket.objects.all():
        filename = s3_object.key
        my_bucket.download_file(s3_object.key, filename)
        
download_all_files()

In [ ]:
!mkdir data

In [ ]:
!mv *.json data/

In [43]:
train_df = pd.read_json('data/train.json').T
train_df.head()

,config,content,conversation_rating
t_bde29ce2-4153-4056-9eb7-f4ad710505fe,C,[{'message': ['Are you a fan of Google or Micr...,"{'agent_1': 'Excellent', 'agent_2': 'Excellent'}"
t_1abc9c37-387d-4013-8691-88ef8c010e58,B,"[{'message': ['do you like dance?'], 'agent': ...","{'agent_1': 'Excellent', 'agent_2': 'Excellent'}"
t_1a600621-5ad4-409c-a812-bc0b2bb03aa6,C,[{'message': ['Hey what's up do use Google ver...,"{'agent_1': 'Excellent', 'agent_2': 'Excellent'}"
t_01269680-99c3-4ab4-9df3-23901e0623c9,C,"[{'message': ['Hi!', 'do you like to dance?'],...","{'agent_1': 'Excellent', 'agent_2': 'Excellent'}"
t_c4f84350-a9e8-4928-bde8-5193b62388e0,B,"[{'message': ['do you like dance?'], 'agent': ...","{'agent_1': 'Excellent', 'agent_2': 'Excellent'}"


In [81]:
cont_tmp = train_df.iloc[2]['content']
cont_tmp

[{'message': ["Hey what's up do use Google very often?I really love the company and was surprised to hear that it was founded back in 1998."],
  'agent': 'agent_1',
  'segmented_annotations': [{'da': '<Statement>',
    'gt_ks': {'ds': 'wiki',
     'section': 'FS1',
     'start_index': 479,
     'end_index': 553,
     'score': 0.27}}],
  'gt_turn_ks': {'ds': 'wiki',
   'section': 'FS1',
   'start_index': 479,
   'end_index': 553,
   'score': 0.27}},
 {'message': ['i think everyone must use it daily!',
   'its become ingrained in every day life'],
  'agent': 'agent_2',
  'segmented_annotations': [{'da': '<Statement>',
    'gt_ks': {'ds': 'wiki',
     'section': 'FS2',
     'start_index': 558,
     'end_index': 778,
     'score': 0.03}},
   {'da': '<Statement>',
    'gt_ks': {'ds': 'fun_facts',
     'section': 'FS3',
     'index': 1,
     'score': 0.12}}],
  'gt_turn_ks': {'ds': 'fun_facts',
   'section': 'FS3',
   'index': 1,
   'score': 0.11}},
 {'message': ['Agreed.',
   'The Google he

In [104]:
#https://github.com/alexa/Topical-Chat/blob/master/src/wiki/wiki.json

import json

with open("data/alexa/wiki/wiki.json", "r") as f:
    wiki_data = json.load(f)
    shortened_wiki_lead_section = wiki_data['shortened_wiki_lead_section']
    summarized_wiki_lead_section = wiki_data['summarized_wiki_lead_section']



In [105]:
talking_points = list(shortened_wiki_lead_section.keys())

In [106]:
talking_points[:5]

['A horror film is a film that seeks to elicit fear. Initially inspired by literature from authors like Edgar Allan Poe, Bram Stoker, and Mary Shelley, horror has existed as a film genre for more than a century. The macabre and the supernatural are frequent themes. Horror may also overlap with the fantasy, supernatural fiction, and thriller genres.',
 'A soundtrack, also written sound track, can be recorded music accompanying and synchronized to the images of a motion picture, book, television program, or video game; a commercially released soundtrack album of music as featured in the soundtrack of a film, video, or television presentation; or the physical area of a film that contains the synchronized recorded sound.',
 'An album is a collection of audio recordings issued as a collection on compact disc (CD), vinyl, audio tape, or another medium. Albums of recorded music were developed in the early 20th century as individual 78-rpm records collected in a bound book resembling a photogr

In [101]:
chats_datafiles = glob.glob("./data/alexa/*.json")
chats_df = pd.concat([pd.read_json(fp).T.drop(['config','conversation_rating'], axis=1) 
                      for fp in chats_datafiles], ignore_index=True)
print(chats_df.shape)

(10784, 1)


In [102]:
chats_df.head()

,content
0,[{'message': ['Do you know who Emily Dickson i...
1,[{'message': ['Did you know the richest superh...
2,"[{'message': ['What arts do you enjoy?', 'Musi..."
3,[{'message': ['Are you familiar with summit me...
4,"[{'message': ['Do you watch soccer?'], 'agent'..."


In [103]:
[ic['message'] for ic in chats_df.iloc[0]['content']]

[['Do you know who Emily Dickson is?'],
 ['Emily Dickinson?',
  'The poet?',
  'I do!',
  '"Tell all the truth, but tell it slant" she once said.',
  'Do you like her poetry?'],
 ['Yeah she was an icon she died in 1886 at the tender age of 55.'],
 ['Though she was reclusive, she lived an interesting 55 years.',
  'Do you know much about her life?'],
 ['I did not unfortunately!',
  'I hear over the years she shared at least 250 poems with Susan her close friend before marrying Austin.'],
 ['Yes.',
  'she wrote hundreds and hundreds of poems, but they were locked away in a drawer, and a critic said that there were many arresting phrases, nothing scanned or rhymed properly, and so he declined to help get them published.'],
 ['Did you kow theres a poem when read normally is depressing but when read backward is uplifting?'],
 ['Wow!',
  "That is certainly different than Emily's poetry.",
  "There's such a diversity in poetry.",
  "It's no surprise, since poetry dates back to prehistorical t

In [ ]:
# to make sure you use the latest version of kaggle:
!pip install --upgrade --force-reinstall --no-deps kaggle
# with an older version, not all files get downloaded

In [ ]:
import kaggle

In [ ]:
!kaggle datasets download -d rtatman/ubuntu-dialogue-corpus

In [ ]:
!mkdir data && unzip ubuntu-dialogue-corpus.zip -d data

In [ ]:
!rm ubuntu-dialogue-corpus.zip

[Back to Contents](#Table-of-Contents)

### Chats Data

In [ ]:
# Since data has been split in multiple pickle files due to size, we load it this wa
chats_datafiles = glob.glob("./data/Ubuntu-dialogue-corpus/dialogueText*.csv")
chats_df = pd.concat([pd.read_csv(fp) for fp in chats_datafiles], ignore_index=True)

In [ ]:
chats_df.sample(6)

In [ ]:
chats_df.dropna(inplace=True)
chats_df.drop_duplicates(inplace=True)

In [ ]:
print(chats_df.shape)
chats_df.head()

In [ ]:
min(chats_df.date), max(chats_df.date)

In [ ]:
#ymd = pd.DatetimeIndex(chats_df['date'])
#chats_df['year'] = ymd.year
#chats_df['month'] = ymd.month
#chats_df['day'] = ymd.day

In [ ]:
chats_df = chats_df.sort_values(['folder','dialogueID', 'date'])
chats_df.head(10)

In [ ]:
from_to = chats_df[['from','to']].apply(lambda x: ' '.join(np.sort(x)), axis=0)

In [ ]:
from_to.unique()

In [ ]:
chats_df.drop(['folder', 'date', 'year', 'month', 'day'], axis=1, inplace=True)

In [ ]:
chats_df

[Back to Contents](#Table-of-Contents)

[Back to Contents](#Table-of-Contents)

### Talking Points Data

[Back to Contents](#Table-of-Contents)

[Back to Contents](#Table-of-Contents)

## A First Glance at Transcripts

Is the discussed button a reliable label? 

i=0, wrongly tagged as discussed

i=3 correct

i=4 wrong

i=7 correct

Is Nexidia transcription accurate enough to capture the talking points or are there cases where the transcriptions missed those phrases?

Am I merging Nexus and Nexidia correctly? It seems so since for some cases I see correct results

[Back to Contents](#Table-of-Contents)

## A First Glance at Chats

In [ ]:
chats_df.iloc[:5]

In [ ]:
i = 1
cT = chats_df.iloc[i]['Transcript']
cT = cT.split('\n')

cT

[Back to Contents](#Table-of-Contents)

## Pre-Trained Sentence Encoder

In [ ]:
#https://www.sbert.net/docs/installation.html

#these lines below are example for downloading pre-trained models
#embed_bert = SentenceTransformer('paraphrase-xlm-r-multilingual-v1')
#embed_bert = SentenceTransformer('stsb-roberta-large') #textual similarity

#the models are downloaded to here
#/Users/atambu310/.cache/torch/sentence_transformers/sbert.net_models_paraphrase-xlm-r-multilingual-v1_part
#copy to desired folder, ex: ./models

### Example of Using Encoder

In [107]:
# this is equivalent to Google Universal Sentence Encoder
embed_bert = SentenceTransformer('./models/sbert.net_models_._distiluse-base-multilingual-cased-v2')

# Two lists of sentences
sentences1 = ['The cat sits outside',
             'A man is playing guitar',
             'The new movie is awesome']

sentences2 = ['The dog plays in the garden',
              'A woman watches TV',
              'The new movie is so great']

#Compute embedding for both lists
embeddings1 = embed_bert.encode(sentences1, convert_to_tensor=True)
embeddings2 = embed_bert.encode(sentences2, convert_to_tensor=True)

#Compute cosine-similarits
cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)

#Output the pairs with their score
for i in range(len(sentences1)):
    print("{} \t\t {} \t\t Score: {:.4f}".format(sentences1[i], sentences2[i], cosine_scores[i][i]))

The cat sits outside 		 The dog plays in the garden 		 Score: 0.3065
A man is playing guitar 		 A woman watches TV 		 Score: 0.2188
The new movie is awesome 		 The new movie is so great 		 Score: 0.9821


In [108]:
#lower-casing or space removal are not needed with this model. Apparently, some preproc
#is performed internally to the model

util.pytorch_cos_sim(embed_bert.encode(["With the Wifi Gateway, you'll get access to our simple and digital xFi dashboard."], convert_to_tensor=True), 
                     embed_bert.encode(["with the wi-fi gateway, you will get access to our simple and digital xFi dashboard."], convert_to_tensor=True)).detach().cpu().numpy()

array([[0.97768307]], dtype=float32)

In [39]:
cT = [ic['message'] for ic in chats_df.iloc[0]['content']]
cT

[['Do you know who Emily Dickson is?'],
 ['Emily Dickinson?',
  'The poet?',
  'I do!',
  '"Tell all the truth, but tell it slant" she once said.',
  'Do you like her poetry?'],
 ['Yeah she was an icon she died in 1886 at the tender age of 55.'],
 ['Though she was reclusive, she lived an interesting 55 years.',
  'Do you know much about her life?'],
 ['I did not unfortunately!',
  'I hear over the years she shared at least 250 poems with Susan her close friend before marrying Austin.'],
 ['Yes.',
  'she wrote hundreds and hundreds of poems, but they were locked away in a drawer, and a critic said that there were many arresting phrases, nothing scanned or rhymed properly, and so he declined to help get them published.'],
 ['Did you kow theres a poem when read normally is depressing but when read backward is uplifting?'],
 ['Wow!',
  "That is certainly different than Emily's poetry.",
  "There's such a diversity in poetry.",
  "It's no surprise, since poetry dates back to prehistorical t

In [129]:
scores = []
for ic in cT[:10]:
    #print(ic)
    for it in talking_points:
        #print(it)
        for icc in ic:
            ic_it_dist = util.pytorch_cos_sim(embed_bert.encode([icc], convert_to_tensor=True), 
                                 embed_bert.encode([it], convert_to_tensor=True)).detach().cpu().numpy()
            scores.append(float(ic_it_dist))
    break

In [130]:
np.argmax(scores)

158

In [132]:
scores[158]

0.31129002571105957

In [133]:
talking_points[158]

"Emma Charlotte Duerre Watson (born 15 April 1990) is an English actress, model, and activist. Born in Paris and brought up in Oxfordshire, Watson attended the Dragon School and trained as an actress at the Oxford branch of Stagecoach Theatre Arts. As a child artist, she rose to prominence after landing her first professional acting role as Hermione Granger in the Harry Potter film series, having acted only in school plays previously. Watson appeared in all eight Harry Potter films from 2001 to 2011, earning worldwide fame, critical accolades, and around $60 million.Watson continued to work outside of the Harry Potter films, appearing in the 2007 television adaptation of the novel Ballet Shoes and lending her voice to The Tale of Despereaux (2008). Following the last Harry Potter film, she took on starring and supporting roles in My Week with Marilyn (2011), The Perks of Being a Wallflower (2012) and The Bling Ring (2013), made an appearance as an exaggerated version of herself in This

[Back to Contents](#Table-of-Contents)

## Data Processing

In [ ]:
@ray.remote
class processText:
        
    def __init__(self):
        
        import tensorflow as tf
        import tensorflow_hub as hub
        import tensorflow_text #needed to avoid crashes    
        
        self.texts = ray.get(texts_ray)
        
        print('loading model')
        self.embed = SentenceTransformer('./models/sbert.net_models_._distiluse-base-multilingual-cased-v2')
        print('loading model...done')

        self.talking_points_emb = self.embed.encode(ray.get(talking_points_ray), convert_to_tensor=True)

        
    def clean_sentences(self, cT):

        #create list of senteences from text
        cT = re.sub(r'\.\.\.','<UNK>', cT)
        cT = re.sub(r'<UNK>','', cT)
        cT = cT.split('\n')
                
        cT = [s.lower() for s in cT] #lower case
        cT = [" ".join([w for w in s.split() if len(s.split())>5]) for s in cT]  #extra spaces, at least 5 words

        if len(cT)==0:
            return []

        if len(cT)<3: # at least 3 sentences
            return []
        else:
            return cT
    
    
    def get_points(self, text_idxs, phrase_thr=0.60):

        is_talking_points = {}
        
        print('start', text_idxs)
        
        for i in range(text_idxs[0], text_idxs[1]): 
            
            cT = self.texts[i]
            cT = self.clean_sentences(cT)
            
            if len(cT)==0:
                continue

            emb_cT_i = self.embed.encode(cT, convert_to_tensor=True)
            # similarity 
            sim_mat = util.pytorch_cos_sim(emb_cT_i,
                                           self.talking_points_emb).detach().cpu().numpy()
            
            # loop over individual sentences and check similarity to each of the phrases    
            #best_match = np.unravel_index(np.argmax(sim_mat, axis=None), sim_mat.shape)
            #best_match_value = sim_mat[best_match]
            #if best_match_value>phrase_thr:
            #    is_talking_points.update({(i,cT[best_match[0]]): [(best_match[1], best_match_value)]})
            
            # for each sentence find matching phrases above threshold. 
            # In a chat mutiple talking points can be mentioned
            for j in range(sim_mat.shape[0]):
                sim_idx = np.where(sim_mat[j,]>phrase_thr)[0]
                sim_values = [(k, sim_mat[j,k]) for k in sim_idx] # this is for debugging purposes, later just select best matching phrase
                if len(sim_idx)>0:
                    is_talking_points.update({(i,cT[j]): sim_values})
            

        print('end', text_idxs)
        
        return is_talking_points
    

In [ ]:
ray.shutdown()

### Processing Call Transcripts

In [ ]:
%%time 

num_cpus = psutil.cpu_count(logical=False)
max_calls = 10000#len(calls_df['Transcript'].tolist())
n_cells = 50 #to run in a single process

print('Processing', max_calls, 'calls')

# split task into chunks
run_list_chnks = np.linspace(0, max_calls, int(max_calls/n_cells), dtype=int)
run_list_chnks = [(run_list_chnks[i],run_list_chnks[i+1]) for i in range(0,len(run_list_chnks)-1)]

# init ray
ray.init(num_cpus=num_cpus)
#ray.timeline(filename="./timeline.json") #debugging

phrase_thr = 0.50

texts_ray = ray.put(calls_df['Transcript'].tolist()[:max_calls])
talking_points_ray = ray.put(talking_points)

pT = [processText.remote() for _ in range(num_cpus)]

# every 'num_cpus' jobs, start from worker 0 again
result_ids = [pT[i % num_cpus].get_points.remote(run_list_chnks[i],
                                                 phrase_thr=phrase_thr) for i in range(len(run_list_chnks))] 

# Fetch the results.
#results = ray.get(result_ids)    
results = ray.get(result_ids)    

ray.shutdown()

#### Example of Processed Transcript Data

In [ ]:
results[0] # (transcript index, utterance text above threshold): [(talking point 1, similarity to utterance),(talking point 2, similarity to utterance)] 

In [ ]:
talking_points[11]

[Back to Contents](#Table-of-Contents)

### Processing Chats

In [ ]:
%%time 

num_cpus = psutil.cpu_count(logical=False)
max_chats = 10000# len(chats_df['Transcript'].tolist())
n_cells = 50 #to run in a single process

print('Processing', max_chats, 'chats')

# split task into chunks
run_list_chnks = np.linspace(0, max_chats, int(max_chats/n_cells), dtype=int)
run_list_chnks = [(run_list_chnks[i],run_list_chnks[i+1]) for i in range(0,len(run_list_chnks)-1)]

# init ray
ray.init(num_cpus=num_cpus)

phrase_thr = 0.50

texts_ray = ray.put(chats_df['Transcript'].tolist()[:max_chats])
talking_points_ray = ray.put(talking_points)

pT = [processText.remote() for _ in range(num_cpus)]

# every 'num_cpus' jobs, start from worker 0 again
result_ids = [pT[i % num_cpus].get_points.remote(run_list_chnks[i],
                                                 phrase_thr=phrase_thr) for i in range(len(run_list_chnks))] 

# Fetch the results.
#results = ray.get(result_ids)    
results_c = ray.get(result_ids)

ray.shutdown()

#### Example of Processed Chat Data

In [ ]:
results_c[0] # (transcript index, utterance text above threshold): [(talking point 1, similarity to utterance),(talking point 2, similarity to utterance)] 

In [ ]:
talking_points[5]

[Back to Contents](#Table-of-Contents)

## Analyzing Processed Data

For each talking point in the table below, we randomly sample few transcript extracts to check reliability of extract retrieval approach used in this notebook 

In [ ]:
def analyze_results(results, n_texts, phrase_thr, verbose=True):

    talking_points_cnts = {k: 0 for k in talking_points}

    # loop over processed chunks and extract statistics
    for rr in range(len(results)):
        for chat_no_extract,tp_no_sim in results[rr].items():
            chat_no, chat_extract = chat_no_extract[0], chat_no_extract[1]
            tp_no_sim_sorted = sorted(tp_no_sim, key=lambda x: x[1], reverse=True) # sort by similarity values
            for kk in tp_no_sim_sorted[:1]: # take best
                tp_no = kk[0]
                tp_sim = kk[1]
                if tp_sim>phrase_thr:
                    if verbose:
                        print('------> Chat no. ', chat_no, 'Type: ', df.chatQueueLob[chat_no])
                        print('---> Extract:', chat_extract)
                        print('---> Phrase ', talking_points[tp_no])
                        print('---> Similarity index', tp_sim)
                    talking_points_cnts[talking_points[tp_no]] += 1
                 
    #extract some examples of match
    sample_extracts = {tp: [] for tp in talking_points}
    for tp in talking_points:
        for rr in range(len(results)):
            for chat_no_extract, tp_no_sim in results[rr].items():
                chat_no, chat_extract = chat_no_extract[0], chat_no_extract[1]
                tp_no_sim_sorted = sorted(tp_no_sim, key=lambda x: x[1], reverse=True)
                for kk in tp_no_sim_sorted[:1]: # take best
                    tp_no = kk[0]
                    tp_sim = kk[1]
                    if tp_sim>phrase_thr and tp_no==talking_points.index(tp):
                        sample_extracts[tp].append('**' + chat_extract)
                        #print('----->', tp)
                        #print('---> Extract:', chat_extract)
        if verbose:
            print(tp, len(sample_extracts[tp]), len(sample(sample_extracts[tp],min(20, len(sample_extracts[tp])))))
        sample_extracts[tp] = '\n'.join(sample(sample_extracts[tp],min(20, len(sample_extracts[tp])))) # sample only 5 examples
        #sample_extracts[tp] = '\n'.join(sample_extracts[tp]) # sample only 5 examples
    sample_extracts = [v for k,v in sample_extracts.items()]
    
    ## put in excel
    results_df = pd.DataFrame({'Talking Points': [k for k,v  in talking_points_cnts.items()], 
                  'Perc. Used': [100*v/n_texts for k,v  in talking_points_cnts.items()],
                  'Sample Extracts': sample_extracts,
                  'Human Corrected': [0 for k in talking_points_cnts]})# randomly sample 20 extracts for each talking point and visually inspect how many are right

    results_df.to_excel('extracts.xlsx')

    return results_df

### Analyzing Transcript

In [ ]:
results_df = analyze_results(results, max_chats, phrase_thr=0.6, verbose=False)
results_df

In [ ]:
i = 5
results_df.iloc[i,0], results_df.iloc[i,2].split('\n')

In [ ]:
i = 6
results_df.iloc[i,0], results_df.iloc[i,2].split('\n')

In [ ]:
i = 19
results_df.iloc[i,0], results_df.iloc[i,2].split('\n')

[Back to Contents](#Table-of-Contents)

### Analyzing Chats

In [ ]:
results_c_df = analyze_results(results_c, max_chats, phrase_thr=0.6, verbose=False)
results_c_df

In [ ]:
i = 5
results_c_df.iloc[i,0], results_c_df.iloc[i,2].split('\n')

In [ ]:
i = 7
results_c_df.iloc[i,0], results_c_df.iloc[i,2].split('\n')

[Back to Contents](#Table-of-Contents)

## Interactive Visualization of Results

In [ ]:
embed = SentenceTransformer('./models/sbert.net_models_._distiluse-base-multilingual-cased-v2')

In [ ]:
talking_points_emb = embed.encode(talking_points, convert_to_tensor=True)

In [ ]:
# input text gets extra slash in front of \n, so we added \\n for splitting sentences
def clean_sentences(cT):

        #create list of senteences from text
        cT = re.sub(r'\.\.\.','<UNK>', cT)
        cT = re.sub(r'<UNK>','', cT)
        cT = cT.split('\\n')
                
        cT = [s.lower() for s in cT] #lower case
        cT = [" ".join([w for w in s.split() if len(s.split())>5]) for s in cT]  #extra spaces, at least 5 words

        if len(cT)==0:
            return []

        if len(cT)<3: # at least 3 sentences
            return []
        else:
            return cT
    
    
def get_points(cT, phrase_thr=0.60):

        cT = clean_sentences(cT)
            
        if len(cT)==0:
            return ''
        
        emb_cT = embed.encode(cT, convert_to_tensor=True)
        # similarity 
        sim_mat = util.pytorch_cos_sim(emb_cT,
                                       talking_points_emb).detach().cpu().numpy()
            
        # for each sentence find matching phrases above threshold. 
        for j in range(sim_mat.shape[0]):
            sim_idx = np.where(sim_mat[j,]>phrase_thr)[0]
            sim_values = [(k, sim_mat[j,k]) for k in sim_idx] 
            sim_sorted = sorted(sim_values, key=lambda x: x[1], reverse=True) # sort by similarity values
            if len(sim_sorted)>0:
                print('**utterance:', cT[j])
                print('Talking Point:', talking_points[sim_sorted[0][0]])
                print('Similarity: ', sim_sorted[0][1])            


In [ ]:
print('**Available talking points**')
print('')
for i in talking_points:
    print(i)

In [ ]:
#Chats
i=185 #11, 185, 2010, 2334
chats_df['Transcript'].iloc[i]
#Calls
#i=1460 #856, 865, 1460
#calls_df['Transcript'].iloc[i]

In [ ]:
@interact_manual(in_text='', phrase_thr=widgets.FloatSlider(min=0.5, max=0.8, step=0.1, value=0.5))
def g(in_text, phrase_thr):
    return get_points(in_text, phrase_thr) 

In [ ]:
## add pop-up message saying if talking point is pressed

[Back to Contents](#Table-of-Contents)